In [4]:
import numpy as np
import pandas as pd
from plotnine import *
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['axes.unicode_minus'] = False

from glob import glob
import re
import math

In [5]:
file_list = []
for i in range(2015,2020):
    a = '../../data/team/해양환경측정망_'+str(i)+'.xlsx'
    file_list.append(a)

['../../data/team/해양환경측정망_2015.xlsx',
 '../../data/team/해양환경측정망_2016.xlsx',
 '../../data/team/해양환경측정망_2017.xlsx',
 '../../data/team/해양환경측정망_2018.xlsx',
 '../../data/team/해양환경측정망_2019.xlsx']

In [6]:
df = pd.read_excel('../../data/team/해양환경측정망_2019.xlsx',skiprows = 3, header=None)



In [7]:
# 전처리 1
key = ['2월','5월','8월','11월']
con_data = pd.DataFrame()
for i in file_list : 
    df_all = pd.read_excel(i, skiprows = 3, header=None, sheet_name=None)
    for j in key:
        # 월별시트 호출
        data = df_all[j]
        
        # h제거
        h= data[data[4].str.contains('H', na=False)]
        h[4] = h[4].str.get(i=1)
        h1 = h.index
        for h_1 in range(len(h1)):
            data.loc[h1[h_1],4] = h.loc[h1[h_1],4]
            
        # 항만제거
        man = data[data[2].str.contains('항만', na=False)]
        man[2] = man[2].str[8:-1]
        h2 = man.index
        for h_2 in h2:
            data.loc[h_2,2] = man.loc[h_2,2]
            
        
        # 1로 시작하는 지역의 인덱스 추출
        index = data[data[4] == 1].index
        
        # index, st를 통해 빈칸채우기
        for z in range(len(index)):  
            insert_l = data.loc[index[z],:3]
            
            # 마지막인덱스 이외의 인덱스
            if index[z] < index[len(index)-1]:
                
                # 항만 중 이중으로 열병합 되어있는 경우 0:3까지 nan처리됨
                # nan값은 nan == nan 일시 Flase가 나옴
                # 0:3까지의 값은 이전 인덱스의 값과 동일하기 때문에 이전 인덱스의 값 호출
                if data.loc[index[z],3] != data.loc[index[z],3]:
                    for k in range(index[z],index[z+1]):
                        insert_l_1 = data.loc[index[z]-1,:3]
                        data.loc[k,:3] = insert_l_1
                else :
                    # 1이 연속으로 붙어있는데 한 카테고리일 경우
                    if data.loc[index[z],0] != data.loc[index[z],0]:
                        insert_l_1 = data.loc[index[z]-1,:2]
                        for a_1 in range(index[z],len(data)):
                            data.loc[a_1,:2] = insert_l_1
                    else:
                        
                        for a in range(index[z],index[z+1]):
                            data.loc[a,:3] = insert_l
                        
            # 마지막 인덱스
            else : 
                if data.loc[index[z],3] != data.loc[index[z],3]:
                    for p in range(index[z],len(data)):
                        insert_l_1 = data.loc[index[z]-1,:3]
                        data.loc[p,:3] = insert_l_1
                else:
                    
                    if data.loc[index[z],0] != data.loc[index[z],0]:
                        insert_l_1 = data.loc[index[z]-1,:2]
                        for e_1 in range(index[z],len(data)):
                            data.loc[e_1,:2] = insert_l_1
                    else: 
                        
                        for e in range(index[z],len(data)):
                            data.loc[e,:3] = insert_l
                    
                    
        con_data = pd.concat([con_data, data])


In [8]:
con_data.to_csv('../../data/team/해양환경측정망_합본_R.csv', index=False, encoding = 'ANSI')
con_data.to_csv('../../data/team/해양환경측정망_합본.csv', index=False, encoding = 'utf-8')

In [9]:
df = pd.read_csv('../../data/team/해양환경측정망_합본.csv', encoding='utf-8', engine='python')

index_df = pd.read_excel('../../data/team/해양환경측정망_2019.xlsx')

In [10]:
index_list = index_df.loc[0,:]
for i in range(37):
    df.rename(columns = {str(i): index_list[i]}, inplace = True)
df = df[df['년'] >= 2017]
df = df.dropna()

In [11]:
# 구군추출
# 먼저 연안명칭을 구군에 넣어 빈칸이 없게 만듬
loc_list = df['연안명칭'].unique()
list_name = ['연안','만','하구','만','.자란만','동안','남안','외안','호']
df['구군'] = df['연안명칭']
for j in list_name:
    name_length = df['연안명칭'].str.find(j)
    
    # find는 값이 없으면 -1을 호출, 이를 안하면 중간에 에러발생
    # -1인값은 먼저 넣어둔 연안명칭을 그대로 사용
    for i in name_length.index :
        if name_length[i] != -1:
            df.loc[i,'구군'] = df.loc[i, '연안명칭'][0:name_length[i]]

In [12]:
df.to_csv('../../data/team/해양환경측정망_합본_1_R.csv', index=False, encoding = 'ANSI')
df.to_csv('../../data/team/해양환경측정망_합본_1.csv', index=False, encoding = 'utf-8')

In [ ]:
df = pd.read_csv('../../data/team/해양환경측정망_합본_1.csv')

# 로마자로 기제되어 있는 해수수질기준을 숫자로 변환
class_list = df['해수수질기준'].unique()
a = 1

# 각 로마자를 가지는 인덱스번호를 추출한 뒤 이 값의 해수수질기준에 a값을 넣어줌
for i in class_list:
    index = df[df['해수수질기준']==i].index
    df.loc[index,'해수수질기준']= a
    a += 1
    
for i in range(len(df)):
    if df.loc[i,'생태구역'] =='동 해':
        df.loc[i,'생태구역'] = '동해'

In [14]:
# 형식이 다른  15, 16 년도 자료를 병합
df1 = pd.read_csv('../../data/team/해양환경측정망_15,16.csv')

In [15]:
df = pd.concat([df,df1])
# 자료기입상 발생한 오타같음 7,70 -> 7.70
df.iloc[7952,11] = 7.90

df.to_csv('../../data/team/해양환경측정망_합본_2_R.csv', index=False, encoding = 'ANSI')
df.to_csv('../../data/team/해양환경측정망_합본_2.csv', index=False, encoding = 'utf-8')